In [5]:
# -*- coding: utf-8 -*-
"""
crawl_release_date_v2.ipynb

Đã sửa lỗi và cải thiện:
1. Cài đặt thêm 'tqdm' (thanh tiến trình) và 'lxml' (parser nhanh hơn).
2. Sửa search_release_date() để chỉ tìm trong các snippet kết quả (chính xác hơn).
3. Mở rộng Regex để khớp với nhiều định dạng ngày hơn.
4. Đặt time.sleep(2) vào *bên trong* hàm search để luôn tuân thủ việc delay.
5. Sửa fill_missing_release_dates() để chỉ lặp qua các hàng *bị thiếu* dữ liệu (nhanh hơn).
6. Thêm thanh tiến trình 'tqdm' để xem tiến độ.
7. Sửa logic ghi log vào hàng 501 để đảm bảo an toàn, không ghi đè dữ liệu sản phẩm.
"""

# Commented out IPython magic to ensure Python compatibility.
# %pip install pytz tqdm lxml
from google.colab import files
files.upload()
from IPython.display import clear_output
clear_output()

import pandas as pd
import re
import requests
import random
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pytz
from tqdm.auto import tqdm # Thêm tqdm để tạo thanh tiến trình

def search_release_date(product_name):
    """
    Search release date of product using DuckDuckGo.
    Cải thiện: Chỉ tìm trong snippet và dùng regex tốt hơn.
    """
    query = f"{product_name} release date"
    url = f"https://duckduckgo.com/html/?q={requests.utils.quote(query)}"
    USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/118.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 Version/14 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/117.0.0.0 Safari/537.36"
]

    headers = { "User-Agent": random.choice(USER_AGENTS) }


    # Luôn delay 2 giây để tránh bị block
    time.sleep(random.randint(4, 6))

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status() # Kiểm tra lỗi HTTP (ví dụ: 403, 503)
        soup = BeautifulSoup(response.text, "lxml") # Dùng lxml nhanh hơn

        # Regex pattern mở rộng
        # Thêm: "1 January 2024", "1/1/2024", và "2024" (như một lựa chọn cuối)
        date_pattern = re.compile(
            r"([A-Z][a-z]+ \d{1,2}, \d{4}|\d{1,2} [A-Z][a-z]+ \d{4}|[A-Z][a-z]+ \d{4}|\d{4}-\d{2}-\d{2}|\d{1,2}/\d{1,2}/\d{4}|\b(19[89]\d|20\d{2})\b)"
        )

        # SỬA LỖI QUAN TRỌNG: Chỉ tìm trong các snippet kết quả, không phải toàn bộ trang
        snippets = soup.find_all("div", class_="result__snippet")
        if not snippets:
            # Nếu không tìm thấy snippet, thử tìm trong toàn bộ văn bản như một phương án dự phòng
            text = soup.get_text(" ", strip=True)
            match = date_pattern.search(text)
            if match:
                return match.group(1)

        for snippet in snippets:
            match = date_pattern.search(snippet.get_text())
            if match:
                return match.group(1) # Trả về kết quả đầu tiên tìm thấy

    except Exception as e:
        print(f"Error fetching {product_name}: {e}")

    return None


def fill_missing_release_dates(input_csv, output_csv):
    df = pd.read_csv(input_csv)

    # Xác định cột ngày
    if "releaseDate" in df.columns:
        col = "releaseDate"
    elif "release_date" in df.columns:
        col = "release_date"
    else:
        # THAY ĐỔI: Tự động tạo cột mới nếu không tìm thấy
        print("No release date column found. Creating new column 'release_date'...")
        col = "release_date" # Đặt tên cột mặc định
        df[col] = "" # Tạo cột mới và fill bằng string rỗng

    # Chuẩn hóa cột: fillna và đảm bảo là string
    # Cách này tốt hơn là dùng astype(str) rồi tìm "nan"
    df[col] = df[col].fillna("").astype(str)

    # Tạo một 'mask' (bộ lọc) cho các hàng có ngày bị thiếu
    # Giữ lại logic `isin` của bạn vì nó đã hoạt động sau `astype(str)`
    mask = df[col].isin(["", "nan", "NaN", "None"])

    # Lấy ra các hàng cần cập nhật
    rows_to_update = df[mask]

    # THAY ĐỔI: Kiểm tra nếu tất cả các hàng đều rỗng (trường hợp tạo cột mới)
    total_rows_to_update = len(rows_to_update)
    if total_rows_to_update == 0:
         print("No rows with missing release dates. Nothing to do.")
         # Vẫn tiếp tục để ghi log, không return
    else:
        print(f"Found {total_rows_to_update} rows with missing release dates. Starting search...")


    # Ghi lại thời gian bắt đầu
    gmt_plus_7 = pytz.timezone('Etc/GMT-7')
    datetime_string_start = datetime.now(gmt_plus_7).strftime("%Y-%m-%d %H:%M:%S %Z%z")

    # SỬA LỖI HIỆU SUẤT:
    # Chỉ lặp qua các hàng CẦN cập nhật, sử dụng tqdm để hiển thị thanh tiến trình
    # THAY ĐỔI: Thêm điều kiện 'if total_rows_to_update > 0' để không chạy tqdm nếu không cần
    if total_rows_to_update > 0:
        for i, row in tqdm(rows_to_update.iterrows(), total=total_rows_to_update, desc="Filling dates"):
            product_name = row.get("product_name") or row.get("name") or ""
            if not product_name:
                continue

            date_found = search_release_date(product_name)
            if date_found:
                # Dùng .at[i, col] để cập nhật DataFrame gốc một cách an toàn
                df.at[i, col] = date_found
                print(f"   → Found: {product_name} -> {date_found}") # Thêm print khi TÌM THẤY

    # Ghi lại thời gian kết thúc
    datetime_string_finish = datetime.now(gmt_plus_7).strftime("%Y-%m-%d %H:%M:%S %Z%z")

    # SỬA LỖI GHI LOG:
    # Ghi log vào hàng 501 một cách an toàn
    log_message = f"Execution started at: {datetime_string_start}\nExecution finished at: {datetime_string_finish}"

    # Nếu hàng 501 đã tồn tại (ví dụ: là một sản phẩm), hãy xóa nó
    if 501 in df.index:
        df.loc[501, :] = pd.NA # Xóa tất cả dữ liệu ở hàng 501

    # Thêm/Cập nhật thông tin log vào cột [col] tại hàng 501
    # Thao tác này sẽ TẠO MỚI hàng 501 nếu nó chưa tồn tại
    df.loc[501, col] = log_message

    df.to_csv(output_csv, index=False)
    print(f"\nDone! Saved updated file to {output_csv}")


# Example usage
# Đảm bảo bạn đã upload file 'fix_products.csv'
fill_missing_release_dates("fix_products.csv", "fix_products_filled.csv")

from google.colab import files
files.download('fix_products_filled.csv')

Found 500 rows with missing release dates. Starting search...


Filling dates:   0%|          | 0/500 [00:00<?, ?it/s]

   → Found: iPhone 16 Pro Max 256GB -> 2024-09-09
   → Found: iPhone 17 Pro Max -> 2025
   → Found: iPhone 15 128GB -> 2023
   → Found: Xiaomi 14T Pro 12GB 512GB -> Sep 2024
   → Found: Samsung Galaxy Z Fold7 12GB 256GB -> 2025-07-09
   → Found: Samsung Galaxy S24 Ultra 12GB 256GB -> Jan 2024
   → Found: iPhone 16 128GB -> September 9, 2024
   → Found: OPPO Reno14 F 5G 8GB 256GB -> Jun 2025
   → Found: iPhone 17 -> 2025
   → Found: iPhone 16 Pro 128GB -> 2024-09-13
   → Found: Xiaomi 15T 5G 12GB 512GB -> 2025-09-24
   → Found: iPhone 15 Pro Max 256GB -> 2023
   → Found: Samsung Galaxy S25 Ultra 12GB 256GB -> 2025-03-28
   → Found: Samsung Galaxy S25 FE 8GB 128GB -> 2025-09-04
   → Found: iPhone Air -> 2025
   → Found: iPhone 17 Pro -> 2025
   → Found: Samsung Galaxy Z Flip7 12GB 256GB -> 2025
   → Found: iPhone 14 128GB -> 2022
   → Found: Samsung Galaxy A56 5G 8GB 128GB -> March 2, 2025
   → Found: iPhone 13 128GB -> 2025-08-18
   → Found: iPhone 16 Pro Max 512GB -> 2024-09-09
   → Fo

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>